In [1]:

import wandb
from sumolib import checkBinary
import Genetic_Algorithm as GA
import Simulation_Generator as sg
import numpy as np

(b'<configuration><input><net-file value="Manhattan5x3.net.xml" /><route-files '
 b'value="Manhattan5x3.rou.xml" /></input><time><begin value="0" /><end value="'
 b'200" /></time><time-to-teleport value="-1" /></configuration>')


In [2]:
wandb.init(project="Manhattan 5x3 Genetic Algorithm", entity="parin-p-ahmedabad-university")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: parin-p (parin-p-ahmedabad-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:

#SUMO parameters
sumo_binary = checkBinary('sumo')
sumo_binary_gui = checkBinary('sumo-gui')
config_file = 'Manhattan5x3.sumocfg'

pop_size = 5 # 12
max_generations = 7  # 10
n_survivors = 2 # 5

#Mutation and Crossover parameters
crossover_rate=0.5
duration_mutation_rate=0.2
duration_mutation_strength = 30  # Increase mutation strength
states_mutation_rate = 0.3
light_options = ['G', 'y', 'r']


#Fitness Function parameters
collision_penalty   = [400,4000,10000,400,4000]
waiting_time_weight = [1/200, 3/200, 2/200, 5/200]
emissions_weight    = [10/20000,5/2000,15/2000,0/2000]


# Launch 5 simulated experiments
total_runs = 2
for run in range(total_runs):
  # 🐝 1️⃣ Start a new run to track this script
    wandb.init(
      # Set the project where this run will be logged
      project="Manhattan5x3_GA", 
      entity="dharma",
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"GA_270321_{run}",
      # Track hyperparameters and run metadata
      config={
        "config_file":config_file,
        "collision_penalty": collision_penalty[run],
        "waiting_time_weight": waiting_time_weight[run],
        "emissions_weight": emissions_weight[run],
        "architecture": "Genetic Algorithm",
        "population_size": pop_size,
        "max_generations":max_generations,
        "n_survivors":n_survivors,
        "crossover_rate":crossover_rate,
        "duration_mutation_rate":duration_mutation_rate,
        "duration_mutation_strength":duration_mutation_strength,
        "states_mutation_rate":states_mutation_rate       
      }
    )

    #Get Base chromosome
    chromosome_base = GA.get_chromosome()
    GA_pop_next = GA.generate_random_population(chromosome_base)
    current_gen = 0
    #Generations=[]
    
    #Run GA
    while(current_gen <max_generations):
        GA_pop = GA_pop_next
        
        
        ## Evaluating Population
        fitness, emissions, waiting = GA_pop.evaluate_pop()
        #Generations.append([GA_pop.generation,GA_pop.best_fitness,GA_pop.best_emissions,GA_pop.best_waiting_time,
        #                    GA_pop.avg_fitness,GA_pop.avg_emissions,GA_pop.avg_waiting_time])

        print("Best Fitness value in Generation ",current_gen, " is ",GA_pop.best_fitness)

        #Logging data to Weights and Biases
        best_fitness = GA_pop.best_fitness
        best_emissions = GA_pop.best_emissions
        best_waiting_time = GA_pop.best_waiting_time
        avg_fitness = GA_pop.avg_fitness
        avg_emissions = GA_pop.avg_emissions
        avg_waiting_time = GA_pop.avg_waiting_time

        print("Logging info to WandB")
        wandb.log({"best_fitness":best_fitness,"best_emissions":best_emissions,"best_waiting_time":best_waiting_time,
                   "avg_fitness":avg_fitness, "avg_emissions":avg_emissions,"avg_waiting_time":avg_waiting_time})
        
        
        ## Visualize SUMO run
        if current_gen // 5 == 0:
            GA.visualize_SUMO(GA_pop.best_individual)
        
        #Calculating Survivors
        sorted_pop = np.argsort(fitness)[:n_survivors]
        Next_gene_pool = [GA_pop.gene_pool[idx] for idx in sorted_pop]
        
        #Creating new Generation
        while len(Next_gene_pool) < pop_size:

            chromosome_male =GA_pop.gene_pool[int(np.random.uniform(0,n_survivors))]
            chromosome_female=GA_pop.gene_pool[int(np.random.uniform(0,n_survivors))]
            chromosome_child = GA.crossover_parent(chromosome_male, chromosome_female,crossover_rate)
            chromosome_child = GA.mutate_chromosome(chromosome_child,0.1,5,0.2)
            Next_gene_pool.append(chromosome_child)

        current_gen = current_gen+1
        GA_pop_next = GA.population(current_gen)
        GA_pop_next.gene_pool = Next_gene_pool

    wandb.finish()

SUMO launched: Collecting Traffic Lights
Generation: 0  Individual:  0
SUMO launched
Generation: 0  Individual:  1
SUMO launched
Generation: 0  Individual:  2
SUMO launched
Generation: 0  Individual:  3
SUMO launched
Generation: 0  Individual:  4
SUMO launched
Generation: 0  Individual:  5
SUMO launched
Generation: 0  Individual:  6
SUMO launched
Generation: 0  Individual:  7
SUMO launched
Generation: 0  Individual:  8
SUMO launched
Generation: 0  Individual:  9
SUMO launched
Generation: 0  Individual:  10
SUMO launched
Generation: 0  Individual:  11
SUMO launched
Generation: 0  Individual:  12
SUMO launched
Generation: 0  Individual:  13
SUMO launched
Generation: 0  Individual:  14
SUMO launched
Best Fitness value in Generation  0  is  8853281193
Logging info to WandB
Generation: 1  Individual:  0
SUMO launched
Generation: 1  Individual:  1
SUMO launched
Generation: 1  Individual:  2
SUMO launched
Generation: 1  Individual:  3
SUMO launched
Generation: 1  Individual:  4
SUMO launched


avg_emissions,█▃▄▁▂▁▁
avg_fitness,█▃▄▂▂▁▁
avg_waiting_time,█▃▃▁▁▁▁
best_emissions,███▅▅▄▁
best_fitness,███▄▄▄▁
best_waiting_time,█▇▇▇▇▂▁
avg_emissions,118903307.8
avg_fitness,8821677029.2
avg_waiting_time,346730
best_emissions,118466550
best_fitness,8800260863


CommError: failed to upsert bucket: returned error 404: {"data":{"upsertBucket":null},"errors":[{"message":"project not found","path":["upsertBucket"]}]}

In [5]:
def get_best_light_timings(best_individual):
    """
    Displays the traffic light timings (Red, Yellow, Green) for the best individual 
    in the population based on the genetic algorithm evaluation.
    
    :param best_individual: The best chromosome (individual) from the GA population
    """
    for junction in best_individual:
        ID = junction.ID
        states = junction.states
        durations = junction.durations
        
        print(f"Traffic Light ID: {ID}")
        print(f"{'State':<10} {'Duration (s)'}")
        print("-" * 30)
        
        for state, duration in zip(states, durations):
            print(f"{state:<10} {duration}")
        
        print("-" * 30)
        print("\n")


In [ ]:
GA.visualize_SUMO(GA_pop.best_individual)

NameError: name 'GA' is not defined